In [1]:
import networkx as nx
from tqdm import tqdm
from datetime import datetime
import numpy as np
import random


In [2]:
file = open('ca-AstroPh.txt','r')
lines = file.readlines()

In [3]:
def dic2list(myDic):
    graph = []
    for i in myDic.keys():
        graph.append(i)
    return graph

In [4]:
nodes = {}
unique_nodes = set()
neighbor_c = {}
for l in lines:
    if '#' in str(l):
        continue
    node = l.replace('\n','').split('\t')
    n1 = int(node[0])
    n2 = int(node[1])
    nodes[(n1,n2)] = 1
    unique_nodes.add(n1)
    unique_nodes.add(n2)
    if  neighbor_c.get(n1,0) == 0:
             neighbor_c[n1] = []
    neighbor_c[n1].append(n2)
nodes_set = dic2list(nodes)

In [5]:
def my_bfs(visited, queue, graph, node):
    visited.append(node)
    queue.append(node)
    n_list = []
    while queue:
        s = queue.pop(0)
        n_list.append(s)
        # print(graph[s])
        if graph.get(s,0) != 0:
            for neighbour in graph[s]:
                if neighbour not in visited:
                    visited.append(neighbour)
                    queue.append(neighbour)
    return n_list

In [6]:
def reward(visited, queue, graph, node,out):
    visited.append(node)
    queue.append(node)
    i = 1
    while queue:
        s = queue.pop(0)
        if s == out:
            break
        i+=1
        # print("s",s)
        # for neighbour in graph[s]:
        if graph.get(s,0) != 0:
           for neighbour in graph.get(s,0):
                if neighbour not in visited:
                    visited.append(neighbour)
                    queue.append(neighbour)
    # print(node,out,i)
    return i

In [8]:
def iter_bfs(neigh,neighbor_probab):
    visited_n = []
    queue_n = []
    n_neighbor = {}
    for a in tqdm(neigh):
        n_neighbor[a] = reward(queue_n, visited_n,neighbor_probab,a)
    return n_neighbor

In [9]:
# iter_bfs(unique_nodes,neighbor_c)

In [10]:
def generate_realization_graph(nodes,p):
    neighbor_probable = {}
    nodes_probable_unique = set()
    nodes_probable =  nodes.copy()
    for i in list(nodes_probable.keys()):
        if np.random.uniform(0,1,1)[0] > p:
            del nodes_probable[i]
        else:
            nodes_probable_unique.add(i[0])
            nodes_probable_unique.add(i[1])
            if  neighbor_probable.get(i[0],0) == 0:
                 neighbor_probable[i[0]] = []
            if  neighbor_probable.get(i[1],0) == 0:
                 neighbor_probable[i[1]] = []
            neighbor_probable[i[0]].append(i[1])

    return nodes_probable,nodes_probable_unique,neighbor_probable


In [31]:
nodes_p,nodes_p_unique,neighbor_p =  generate_realization_graph(nodes,0.016)
len(nodes_p)

6296

In [12]:
# print(nodes_p)
# G = nx.DiGraph()
# G.add_edges_from(dic2list(nodes_p))
# nx.draw(G,with_labels=True)

In [32]:
r_nodes_graph = list(nodes_p_unique)
random.shuffle(r_nodes_graph)
outbreak = r_nodes_graph[0:10]
print(outbreak)

[115331, 127219, 131654, 77819, 29959, 119007, 122336, 29880, 5883, 7019]


In [57]:
outbreack_neighbor = {}
def outbreak_out(outbreak):
    for o in outbreak:
        outbreack_neighbor[o] = my_bfs([],[],neighbor_p,o)
outbreak_out(outbreak)
outbreack_neighbor

{115331: [115331],
 127219: [127219,
  61358,
  65467,
  27943,
  108753,
  125316,
  100021,
  2962,
  31756,
  85451,
  4148,
  88596,
  101975,
  107846,
  88102,
  24751,
  36148,
  75314,
  94624,
  98981,
  8995,
  53342,
  113903,
  75612,
  110749,
  24593,
  80692,
  81790,
  84844,
  100181,
  34454,
  120556,
  120249,
  81792],
 131654: [131654, 40419, 45839],
 77819: [77819],
 29959: [29959],
 119007: [119007],
 122336: [122336],
 29880: [29880, 13152, 112779, 90535, 18977],
 5883: [5883, 48956],
 7019: [7019, 43032]}

In [55]:
rewards = {}
def calculate_node_reward():
    for k,v in neighbor_p.items() :
        if len(v) == 0:
            continue
        n_list = []
        for o in outbreak:
            if k in outbreack_neighbor[o]:
                n_list.append(-1 * reward([],[],neighbor_p,o,k))
            else:
                n_list.append(0)
            # print(k,n_list)
        if np.sum(n_list) != 0 and k not in outbreak:
            rewards[k] = np.sum(n_list)
        print(k,n_list)

calculate_node_reward( )
# print(rewards)


84424 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
76259 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
94138 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
127393 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
106274 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17097 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
54239 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
71482 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
76830 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
76529 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
105725 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13353 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
87885 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4959 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
35690 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10426 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
126916 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
28577 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
29489 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
23469 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
64650 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
101811 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17689 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
8273 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
837 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
48703 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
70099 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [35]:
sorted(rewards, key=lambda k: rewards[k], reverse=True)[0:10]

[13152, 61358, 27943, 90535, 108753, 125316, 100021, 2962, 31756, 85451]

### Greedy

In [52]:
def calculate_nodes_reward(neighbor_p,outbreak ):
    reward_new = {}
    for k,v in tqdm(neighbor_p.items()) :
        if len(v) == 0:
            continue
        n_list = []
        for o in outbreak:
            if k in outbreack_neighbor[o]:
                n_list.append(-1 * reward([],[],neighbor_p,o,k))
            else:
                n_list.append(0)
             # n_list.append(-1 * reward([],[],neighbor_p,o,k))
        if np.sum(n_list) != 0:
            reward_new[k] = np.sum(n_list)
        if k == 48956:
            print(k,n_list)
    return reward_new



In [59]:
sort = sorted(rewards, key=lambda k: rewards[k], reverse=True)
def hill_climing():
    S={}
    cost = []
    reward_h = rewards.copy()
    neighbor_p_c = neighbor_p.copy()
    for i in outbreak:
        del neighbor_p_c[i]
    cost.append(len(neighbor_p_c[sort[0]]) * 0.8)
    S[sort[0]] = reward_h[sort[0]]
    outbreak_h = outbreak.copy()

    outbreak_h.pop(0)
    neigh = set()
    for s in S:
        del neighbor_p_c[s]
    while np.sum(cost) < 10:
        temp_dic = {}
        temp_dic.update(calculate_nodes_reward(neighbor_p_c,outbreak_h))
        if len(temp_dic) == 0:
            break
        n = sorted(temp_dic, key=lambda k: temp_dic[k], reverse=True)[0]
        S[n] = temp_dic[n]
        del neighbor_p_c[n]
        outbreak_h.pop(0)
        cost.append(len(neighbor_p[n]) * 0.8)
    return S
print(hill_climing())

100%|██████████| 6650/6650 [00:00<00:00, 250821.66it/s]

{13152: -2, 85451: -2, 90535: -2}


In [ ]:
# len(list(nx.bfs_tree(nx.DiGraph(dic2list(nodes_p)),3  )))

### Lazy Hill Climbing

In [ ]:
mg_sorted = sorted(rewards, key=lambda k: rewards[k], reverse=True)
mg_sorted[0:10]

In [ ]:
def calculate_node_reward_hill(node,neighbor_p,outbreak ):
    reward_new = {}

    n_list = []
    for o in outbreak:
        if node in outbreack_neighbor[o]:
                n_list.append(-1 * reward([],[],neighbor_p,o,node))
        else:
                n_list.append(0)

    reward_new[node] = np.sum(n_list)
    return reward_new

In [ ]:
def celf_mg():
    S = {}
    mg_lh_sorted = mg_sorted.copy()
    rewards_lhc = rewards.copy()
    # S.append(mg_lh_sorted[0])
    S[mg_lh_sorted[0]] = rewards_lhc[mg_lh_sorted[0]]
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    neighbor_p_c = neighbor_p.copy()

    outbreak_h.pop(0)
    for i , n in enumerate(mg_lh_sorted):
        if len (S) > 9:
            break
        left = n
        right = mg_lh_sorted[i+1]
        l = rewards_lhc[left]
        r = rewards_lhc[right]
        if True :
            if i != 0:
                # S.append(n)
                S[n] = rewards_lhc[n]

            rewards_lhc[mg_lh_sorted[i+1]] = calculate_node_reward_hill(mg_lh_sorted[i+1],neighbor_p_c,outbreak_h)
            unique_nodes_h.remove(n)
            del neighbor_p_c[n]
        else:
            neighbor_p_c = sorted(rewards, key=lambda k: rewards[k], reverse=True)
    return S


celf_I = celf_mg()
celf_I

### Celf with lazy hill climbing (benefit cost version)

In [ ]:
def calculate_benefti_cost(node,neighbor_p,outbreak ):
    reward_new = {}
    cost = {}
    n_list = []
    for o in outbreak:
        if node in outbreack_neighbor[o]:
                n_list.append(-1 * reward([],[],neighbor_p,o,node))
        else:
                n_list.append(0)

    reward_new[node] = (np.sum(n_list) * 0.25 * -1)/(0.8* len(neighbor_p[node]))
    cost[node] = (0.8* len(neighbor_p[node]))
    return reward_new,cost

In [ ]:
def calculate_node_reward(outbreak ):
    rewards_bc = {}
    cost = {}
    for k,v in tqdm(neighbor_p.items()) :
        if len(v) == 0:
            continue
        n_list = []
        for o in outbreak:
            if k in outbreack_neighbor[o]:
                n_list.append(reward([],[],neighbor_p,o,k))
            else:
                n_list.append(0)

        rewards_bc[k] = (np.sum(n_list) * 0.25 * -1)/0.8* len(v)
        cost[k] = 0.8* len(v)
        # print(k,v,len(v))
        # rewards[n] = np.sum(n_list)
    return rewards_bc,cost

x= calculate_node_reward( outbreak)
# print(x)

In [ ]:
mg_celf_bc_sorted = sorted(x[0], key=lambda k: x[0][k], reverse=True)
def celf_bc():
    S = {}
    rewards_lhc = x[0].copy()
    mg_lh_sorted = mg_celf_bc_sorted.copy()
    S[mg_lh_sorted[0]] = rewards_lhc[mg_lh_sorted[0]]
    unique_nodes_h = nodes_p_unique.copy()
    outbreak_h = outbreak.copy()
    neighbor_p_c = neighbor_p.copy()

    outbreak_h.pop(0)
    for i , n in enumerate(mg_lh_sorted):
        if len (S) > 9:
            break
        left = n
        right = mg_lh_sorted[i+1]
        l = rewards_lhc[left]
        r = rewards_lhc[right]
        if True :
            if i != 0:
                # S.append(n)
                S[n] = rewards_lhc[n]
            rewards_lhc[mg_lh_sorted[i+1]] = calculate_benefti_cost(mg_lh_sorted[i+1],neighbor_p_c,outbreak_h)
            unique_nodes_h.remove(n)
            del neighbor_p_c[n]
        else:
            neighbor_p_c = sorted(rewards_lhc, key=lambda k: rewards_lhc[k], reverse=True)
    return S


celf_II = celf_bc()
celf_II